# Uptake Reactions
This notebook adds uptake reactions for [phenol](https://www.genome.jp/dbget-bin/www_bget?C00146), [vanillate](https://www.genome.jp/dbget-bin/www_bget?C06672), [4-hydroxybenzoate](https://www.genome.jp/dbget-bin/www_bget?C00156), [guaiacol](https://www.genome.jp/dbget-bin/www_bget?C01502), and [benzoate](https://www.genome.jp/dbget-bin/www_bget?C00180) to the *R.opacus* genome scale model.

### The model was made using CarveMe and the commands:
The .faa assembly file was downloaded from [NCBI](https://www.ncbi.nlm.nih.gov/assembly/GCF_020542785.1). 
<ol>
    <li>carve r_opacus_bologna.faa -u grampos -o r_opacus_bologna_raw.xml</li>
    <li>gapfill r_opacus_bologna.xml -m M9,LB -o r_opacus_bologna_gapfilled.xml</li>
</ol>


###  Method
<ol>
    <li>Add metabolites neccessary for aromatic growth</li>
    <li>Add reactions to connect the metabolites to the metabolic network</li>
</ol>

###  Required Additional Metabolites
<ul>
    <li>Phenol: extracellular phenol and cytosolic phenol</li>
    <li>Vanillate: extracellular vanillate</li>
    <li>4-Hydroxybenzoate: none</li>
    <li>Guaiacol: extracellular guaiacol and cytosolic guaiacol</li>
    <li>Benzoate: none</li>
</ul>

###  Required Additional Reactions
<ul>
    <li>Phenol: phenol exchange, phenol transport, and phenol monooxygenase</li>
    <li>Vanillate: vanillate exchange, vanillate transport</li>
    <li>4-Hydroxybenzoate: none</li>
    <li>Guaiacol: guaiacol exchange and guaiacol transport, and guaicaol O-demethylase</li>
    <li>Benzoate: none</li>
</ul>

In [1]:
import cobra

### Load genome-scale model
Get *R.opacus* NCBI model generated by CarveMe

In [2]:
model = cobra.io.read_sbml_model("../GSMs/r_opacus_bologna_gapfilled.xml")
model

Name,italian_3
Memory address,0x07efe908c9210
Number of metabolites,1947
Number of reactions,3011
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


### Add neccessary metabolites to the model

In [4]:
# Extracellular phenol 
phenol_e = cobra.Metabolite(
    'phenol_e',
    formula='C6H6O',
    name='Phenol',
    compartment='C_e')

# Cytosolic phenol
phenol_c = cobra.Metabolite(
    'phenol_c',
    formula='C6H6O',
    name='Phenol',
    compartment='C_c')

# Extracellular 4-hydroxybenzoate
hbz_e = cobra.Metabolite(
    '4hbz_e',
    formula='C7H5O3',
    name='4-Hydroxybenzoate',
    compartment='C_e')

# Extracellular guaiacol
guaiacol_e = cobra.Metabolite(
    'guaiacol_e',
    formula='C7H8O2',
    name='Guaiacol',
    compartment='C_e')

# Cytosolic guaiacol
guaiacol_c = cobra.Metabolite(
    'guaiacol_c',
    formula='C7H8O2',
    name='Guaiacol',
    compartment='C_c')

# Add a metabolites for uptake pathway of vanillate and hydroxybenzoate

# 4-Carboxymuconolactone
cml_4 = cobra.Metabolite(
    '4cml_c',
    formula='C7H4O6',
    name='4-Carboxymuconolactone',
    compartment='C_c')

Add reactions for iron(II) ion uptake

In [4]:
# fe2_transport = cobra.Reaction('fe2_transport')
# fe2_transport.name = 'fe2 transport'
# fe2_transport.lower_bound = 0. 
# fe2_transport.upper_bound = 1000. 
# fe2_transport.add_metabolites({
#     fe2_e: -1.0,
#     model.metabolites.get_by_id('fe2_c'):  1.0,
# }) 

# model.add_reactions([fe2_exchange])
# model.add_reactions([fe2_transport])

Add reactions for phenol exchange, transport, and [conversion to catechol](https://www.kegg.jp/entry/R10043) to the model

In [5]:
phenol_exchange = cobra.Reaction('EX_phenol_e')
phenol_exchange.name = 'Phenol exchange'
phenol_exchange.lower_bound = 0.  
phenol_exchange.upper_bound = 1000.  
phenol_exchange.add_metabolites({
    phenol_e: -1.0,
})

phenol_transport = cobra.Reaction('phenol_transport')
phenol_transport.name = 'Phenol transport'
phenol_transport.lower_bound = 0. 
phenol_transport.upper_bound = 1000. 
phenol_transport.add_metabolites({
    phenol_e: -1.0,
    phenol_c:  1.0,
}) 

# EC 1.14.13.244
phenol_monooxygenase = cobra.Reaction('PHEMOX')
phenol_monooxygenase.name = 'Phenol monooxygenase'
phenol_monooxygenase.lower_bound = 0.
phenol_monooxygenase.upper_bound = 1000.  # This is the default
phenol_monooxygenase.add_metabolites({
    phenol_c: -1.0,
    model.metabolites.get_by_id('nadh_c'):   -1.0,
    model.metabolites.get_by_id('h_c'): -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('nad_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
}) 


model.add_reactions([phenol_exchange])
model.add_reactions([phenol_transport])
model.add_reactions([phenol_monooxygenase])

Add reactions to exchange and transport vanillate

In [6]:
# vanlt_exchange = cobra.Reaction('EX_vanlt_e')
# vanlt_exchange.name = 'Vanillate exchange'
# vanlt_exchange.lower_bound = -1000.  
# vanlt_exchange.upper_bound = 1000.  
# vanlt_exchange.add_metabolites({
#     vanlt_e: -1.0,
# })

# vanlt_transport = cobra.Reaction('vanlt_transport')
# vanlt_transport.name = 'Vanillate transport'
# vanlt_transport.lower_bound = -1000. 
# vanlt_transport.upper_bound = 1000. 
# vanlt_transport.add_metabolites({
#     vanlt_e: -1.0,
#     vanlt_c: 1.0
# }) 

# model.add_reactions([vanlt_exchange])
# model.add_reactions([vanlt_transport])

Add reactions for exchange and transport of hydroxybenzoate

In [7]:
# hbz_exchange = cobra.Reaction('EX_4hbz_e')
# hbz_exchange.name = 'Hydroxybenzoate exchange'
# hbz_exchange.lower_bound = -1000.  
# hbz_exchange.upper_bound = 1000.  
# hbz_exchange.add_metabolites({
#     hbz_e: -1.0,
# })

# hbz_transport = cobra.Reaction('4_hydroxybenzoate_transport')
# hbz_transport.name = 'Hydroxybenzoate transport'
# hbz_transport.lower_bound = -1000. 
# hbz_transport.upper_bound = 1000. 
# hbz_transport.add_metabolites({
#     hbz_e: -1.0,
#     model.metabolites.get_by_id('4hbz_c'):  1.0,
# }) 


# model.add_reactions([hbz_exchange])
# model.add_reactions([hbz_transport])

4_hydroxybenzoate_transportAdd guaiacol reactions for exchange, transport, and [conversion to catechol](https://www.genome.jp/entry/R12301) 

In [6]:
guaiacol_exchange = cobra.Reaction('EX_guaiacol_e')
guaiacol_exchange.name = 'Guaiacol exchange'
guaiacol_exchange.lower_bound = -1000.  
guaiacol_exchange.upper_bound = 1000.  
guaiacol_exchange.add_metabolites({
    guaiacol_e: -1.0,
})

guaiacol_transport = cobra.Reaction('guaiacol_transport')
guaiacol_transport.name = 'Guaiacol transport'
guaiacol_transport.lower_bound = -1000. 
guaiacol_transport.upper_bound = 1000. 
guaiacol_transport.add_metabolites({
    guaiacol_e: -1.0,
    guaiacol_c:  1.0,
}) 

# EC 1.14.14.-
guaiacol_o_demethylase = cobra.Reaction('GUADEM')
guaiacol_o_demethylase.name = 'Guaicaol O-demethylase'
guaiacol_o_demethylase.lower_bound = 0.
guaiacol_o_demethylase.upper_bound = 1000.  # This is the default
guaiacol_o_demethylase.add_metabolites({
    guaiacol_c: -1.0,
    model.metabolites.get_by_id('nadph_c'):   -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('fald_c'):  1.0,
    model.metabolites.get_by_id('nadp_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
    
}) 

model.add_reactions([guaiacol_exchange])
model.add_reactions([guaiacol_transport])
model.add_reactions([guaiacol_o_demethylase])

Add reactions needed for benzoate consumption

In [9]:
# bz_exchange = cobra.Reaction('EX_bz_e')
# bz_exchange.name = 'Benzoate exchange'
# bz_exchange.lower_bound = -1000.  
# bz_exchange.upper_bound = 1000.  
# bz_exchange.add_metabolites({
#     bz_e: -1.0,
# })

# bz_transport = cobra.Reaction('bz_transport')
# bz_transport.name = 'Benzoate transport'
# bz_transport.lower_bound = -1000. 
# bz_transport.upper_bound = 1000. 
# bz_transport.add_metabolites({
#     bz_e: -1.0,
#     model.metabolites.get_by_id('bz_c'):  1.0,
# }) 


# model.add_reactions([bz_exchange])
# model.add_reactions([bz_transport])

Add reactions for the protocatechuate uptake pathway of vanillate and 4-hydroxybenzoate

In [37]:
test_rxn = '4CMLCL_kt'
for r in model.reactions:
    if r.id == test_rxn:
        print(test_rxn, 'is in the model')

4CMLCL_kt is in the model


In [10]:
# VNTDM: h_c + nadh_c + o2_c + vanlt_c --> 34dhbz_c + fald_c + h2o_c + nad_c
# https://www.kegg.jp/entry/1.14.13.82+2.1.1.341
# VNTDM = cobra.Reaction('VNTDM')
# VNTDM.name = 'vanillate monooxygenase'
# VNTDM.lower_bound = 0.  
# VNTDM.upper_bound = 1000.  
# VNTDM.add_metabolites({
#     model.metabolites.get_by_id('h_c'): -1.0,
#     model.metabolites.get_by_id('nadh_c'): -1.0,
#     model.metabolites.get_by_id('o2_c'): -1.0,
#     model.metabolites.get_by_id('vanlt_c'): -1.0,
#     Dihydroxybenzoate_34: 1.0,
#     model.metabolites.get_by_id('fald_c'): 1.0,
#     model.metabolites.get_by_id('h2o_c'): 1.0,
#     model.metabolites.get_by_id('nad_c'): 1.0,
# })

# 4HBHYOX: 4hbz_c + h_c + nadph_c + o2_c --> 34dhbz_c + h2o_c + nadp_c
# https://www.kegg.jp/entry/1.14.13.33+R01296+R01298
HBHYOX = cobra.Reaction('HBHYOX')
HBHYOX.name = '4-hydroxybenzoate 3-monooxygenase'
HBHYOX.lower_bound = 0. 
HBHYOX.upper_bound = 1000. 
HBHYOX.add_metabolites({
    model.metabolites.get_by_id('4hbz_c'): -1.0,
    model.metabolites.get_by_id('h_c'): -1.0,
    model.metabolites.get_by_id('nadph_c'): -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    Dihydroxybenzoate_34: 1.0,
    model.metabolites.get_by_id('h2o_c'): 1.0,
    model.metabolites.get_by_id('nadp_c'): 1.0,
}) 

# PCADYOX: 34dhbz_c + o2_c --> CCbuttc_c + 2.0 h_c
# https://www.genome.jp/dbget-bin/www_bget?ec:1.13.11.3
# PCADYOX = cobra.Reaction('PCADYOX')
# PCADYOX.name = 'protocatechuate 3,4-dioxygenase'
# PCADYOX.lower_bound = 0. 
# PCADYOX.upper_bound = 1000. 
# PCADYOX.add_metabolites({
#     Dihydroxybenzoate_34: -1.0,
#     model.metabolites.get_by_id('o2_c'): -1.0,
#     CCbuttc_c: 1.0,
#     model.metabolites.get_by_id('h_c'): 2.0,
# }) 

# MUCCY_kt: CCbuttc_c + h_c --> 4cml_c
# https://www.kegg.jp/entry/5.5.1.2
# MUCCY_kt = cobra.Reaction('MUCCY_kt')
# MUCCY_kt.name = '3-carboxy-cis,cis-muconate cycloisomerase'
# MUCCY_kt.lower_bound = -1000. 
# MUCCY_kt.upper_bound = 1000. 
# MUCCY_kt.add_metabolites({
#     CCbuttc_c: -1.0,
#     model.metabolites.get_by_id('h_c'): -1.0,
#     cml_4: 1.0
# }) 

# 4CMLCL_kt: 4cml_c + h_c --> 5odhf2a_c + co2_c
# CMLCL_kt = cobra.Reaction('4CMLCL_kt')
# CMLCL_kt.name = '4-carboxymuconolactone decarboxylase'
# CMLCL_kt.lower_bound = -1000. 
# CMLCL_kt.upper_bound = 1000. 
# CMLCL_kt.add_metabolites({
#     cml_4: -1.0,
#     model.metabolites.get_by_id('h_c'): -1.0,
#     model.metabolites.get_by_id('5odhf2a_c'): 1.0,
#     model.metabolites.get_by_id('co2_c'): 1.0
# }) 

# model.add_reactions([VNTDM])
model.add_reactions([HBHYOX])
# model.add_reactions([PCADYOX])
# model.add_reactions([MUCCY_kt])
# model.add_reactions([CMLCL_kt])

#### Define minimal media where the only carbon containing inputs are carbon sources

In [7]:
model.medium = {
 'EX_glc__D_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
#  'EX_phenol_e': 1000.0,
#  'EX_vanlt_e': 1000.0,
#  'EX_guaiacol_e': 1000.0,
#  'EX_tag': 1000.0
}

Confirm the model can grow with glucose

In [8]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of glucose can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
10.0 EX_glc__D_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of glucose can make 0.684 g of biomass


In [13]:
# initial model can NOT grow with phenol
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 10.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of phenol can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
1000.0 EX_fe2_e
10.0 EX_phenol_e
10 mmol of phenol can make 0.000 g of biomass


#### The model can grow if all nonboundary reactions are made reversible

In [14]:
# tests if a reaction contains a metabolite not in the cytosol. These reactions are not boundary reactions
def contains_noncytosolic_metabolite(r_id):
    r = model.reactions.get_by_id(r_id)
    
    # all boundary reactions should not be made reversible
    if r.boundary:
        return True
    
    # check each metabolite to see if it is in the extracellular space or periplasm
    for m in r.metabolites:
        if m.compartment == 'C_e' or m.compartment == 'C_p':
            return True
        
    # return false if all metabolites are in cytosol
    return False

In [15]:
# define a list of non boundary reactions
nonboundary_rxns = [r for r in model.reactions if not contains_noncytosolic_metabolite(r.id)]

print(f'{len(nonboundary_rxns)} of {len(model.reactions)} reaction are nonboundary reactions and may be reversible')

# define list of reactions that can not be reversible
irreversible_reactions = ['ATPM']

with model:
    # make all non boundary reactions reversible
    for r in nonboundary_rxns:
        if r.id not in irreversible_reactions:
            r.lower_bound, r.upper_bound = -10, 10
    
    # define medium to have phenol as sole carbon source
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 10.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    
    # calculate flux solution and save it to a variable
    solution = model.optimize()
    phenol_solution_fluxes = solution.fluxes
    
    print(f'\n10 mmol of phenol can make {solution.objective_value:.3f} g of biomass')

970 of 1761 reaction are nonboundary reactions and may be reversible
1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
1000.0 EX_fe2_e
10.0 EX_phenol_e

10 mmol of phenol can make 1.545 g of biomass


#### Determine the minimum set of reactions that need to be made reversible to allow for growth with phenol

In [16]:
# define a list to store the reactions whose flux is outside the bounds in the previous flux solution
possible_reversible_reactions = []

# loop over all the flux values in the phenol solution that has growth
for r_id, flux_value in phenol_solution_fluxes.items():
    
    r = model.reactions.get_by_id(r_id)
    
    # if the solution flux for a reaction is outside of its current bounds, add it to the possibly reversible reactions list
    if flux_value < r.lower_bound or flux_value > r.upper_bound:
        possible_reversible_reactions.append(r_id)
        
print(f'There are {len(possible_reversible_reactions)} reactions that could to be changed')

There are 107 reactions that could to be changed


The goal of this cell is to determine which reactions NEED to be reversible to allow for growth on phenol.

The model is checked to see if it can grow when all but one of the possible reactions reversible are reversible.

If the model is able to grow when a possible reversible reaction is irreversible then the reaction is REMOVED from list of possible reversible reactions.

If the model is NOT able to grow when a possible reversible reaction is irreversible then the reaction is added to the list of required reversible reactions.

The length of the list of possible reversible reactions is reduced each time all the possible reversible reactions are checked.

The process is repeated until all of the reactions in the possible reversible reactions are required for growth. 

In [17]:
# store list of reactions that need to reversible for growth with phenol
required_reversible_reactions = []

# initialize the solution biomass fluxes list in a way to trigger the while loop.
solution_biomass_fluxes = [1]

# repeat until all the reactions in the possible reversible reactions are required to be reversible for growth
while any(i != 0 for i in solution_biomass_fluxes):
    
    print(f'There are {len(possible_reversible_reactions)} possible reversible reactions')
    
    # for each reaction id in the list of possible reverse reactions
    for possible_reversible_reaction_id in possible_reversible_reactions:

        # reset the list of solution biomass flux values
        solution_biomass_fluxes = []

        with model:

            # make all reactions that are potentially reversible reversible except for the reaction being tested
            for r_id in possible_reversible_reactions:
                if r_id != possible_reversible_reaction_id:
                    r = model.reactions.get_by_id(r_id)
                    r.lower_bound, r.upper_bound = -10, 10

            # define medium to have phenol as sole carbon source
            medium = model.medium
            medium["EX_glc__D_e"] = 0.0
            medium["EX_phenol_e"] = 10.0
            medium["EX_vanlt_e"] = 0.0
            medium["EX_4hbz_e"] = 0.0
            medium["EX_guaiacol_e"] = 0.0
            medium["EX_bz_e"] = 0.0
            model.medium = medium

            # calculate flux solution and print biomass flux when the tested reaction is made irreversible
            solution = model.optimize()

            # add the solution value to the list
            solution_biomass_fluxes.append(solution.objective_value)

            # if the model is able to grow when the tested reaction is not reversible, remove the reaction from the possible reversible list
            if solution.objective_value > 0.0001:
                possible_reversible_reactions.remove(possible_reversible_reaction_id)
                
            # if the model is NOT able to grow when the tested reaction is not reversible, add the reaction to the required reversible list
            else:
                if possible_reversible_reaction_id not in required_reversible_reactions:
                    required_reversible_reactions.append(possible_reversible_reaction_id)

# print the reactions that are required to be reversible for the model to grow
print(f'The required reversible reactions are {required_reversible_reactions}')

There are 107 possible reversible reactions
There are 56 possible reversible reactions
There are 32 possible reversible reactions
There are 20 possible reversible reactions
There are 14 possible reversible reactions
There are 12 possible reversible reactions
There are 10 possible reversible reactions
The required reversible reactions are ['2INSD', 'D5KGPA', 'DKDID', 'MI1PS', 'MMSAD3', 'D5KGK', 'GLYALDDr', 'INS2D', 'MI1PP', 'LACZ']


#### Update the reversibility of the required reversible reactions

In [18]:
for r_id in required_reversible_reactions:
    r = model.reactions.get_by_id(r_id)
    r.lower_bound = -1000
    r.upper_bound = 1000

#### Check that model can grow with each carbon source

In [19]:
# glucose
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of glucose can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe3_e
10.0 EX_glc__D_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
1000.0 EX_fe2_e
10 mmol of glucose can make 1.545 g of biomass


In [20]:
# phenol
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 10.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of phenol can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
1000.0 EX_fe2_e
10.0 EX_phenol_e
10 mmol of phenol can make 1.545 g of biomass


In [21]:
# vanillate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 10.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of vanillate can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
1000.0 EX_fe2_e
10.0 EX_vanlt_e
10 mmol of vanillate can make 2.060 g of biomass


In [23]:
# 4-hydroxybenzoate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 10.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of 4-hydroxybenzoate can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
1000.0 EX_fe2_e
10.0 EX_4hbz_e
10 mmol of 4-hydroxybenzoate can make 1.803 g of biomass


In [24]:
# guaiacol
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 10.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
1000.0 EX_fe2_e
10.0 EX_guaiacol_e
10 mmol of guaiacol can make 1.803 g of biomass


In [25]:
# benzoate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 10.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of benzoate can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
1000.0 EX_fe2_e
10.0 EX_bz_e
10 mmol of benzoate can make 1.803 g of biomass


#### Save model

In [26]:
model.id = 'ropacus_model_A'
model.name = 'R. opacus model A'
cobra.io.write_sbml_model(model, "../GSMs/ropacus_model_A.xml")
model

Name,ropacus_model_A
Memory address,0x07f66a61f33d0
Number of metabolites,1237
Number of reactions,1761
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"
